# Sample Complexity Gap

This notebook aims to demonstrate the stated sample complexity gap in **Why Are Convolutional Networks More Sample Efficient Than Fully-Connected Nets? by Zhiyuan Li, Yi Zhang and Sanjeev Arora** [1]. We set up an experiment in which we should see the gap as an increasing polynomial curve of degree less than two.

## 1. Methods

For a given input dimension $d$, we seek the number $|S_{tr}|$ of training samples needed for a model to reach $\epsilon=0.9$ test accuracy. Then we plot the difference of training samples needed between a Convolutional Neural Network and a Fully Connected Neural Network for increasing values of $d$.

### Data

The inputs are $3\times k \times k$ RGB images for $k\in \mathbb{N}$, yielding input dimensions $d\in \{3,12,27,48,75,108,147,243,300,...\}$. We create full training set of $10'000$ images and a test set of $10'000$ and we ask "the first *how-many* training samples are needed to reach $90\%$ test accuracy if we train until convergence"? The training sets are constructed in the following manner.
+ Entry-wise independent Gaussian (mean 0, standard deviation 1)

We explore two different labelling functions 
\begin{equation}
h_1=\mathbb{1}[\sum_{i\in R} x_i > \sum_{i \in G}x_i] \quad\mathrm{ and }\quad h_2=\mathbb{1}[\sum_{i\in R} x_i^2 > \sum_{i \in G}x_i^2].
\end{equation}

### Models

1. 2-layer CNN.
    + Convolution - one kernel per input channel of size 3x3, 10 output channels, stride size 1, and padding of 1, and bias
    + Activation function
    + Pooling: Max pooling, kernel size 2x2, stride 2
    + Flattening
    + Fully connected layer (? in, 1 out) with bias
    + Sigmoid
2. 2-layer FCNN 
    + Fully connected layer (192 in, 3072 out) with bias
    + Activation function 
    + Fully connected layer (3072 in, 1 out) with bias
    + Sigmoid
    
We try both ReLU and Quadratic activation functions. 

### Training algorithm
+ Stochastic Gradient Descent with batch size 64
+ BCELoss
+ Learning rate $\gamma = 0.01$
+ Stopping criterion: At least 10 epochs AND Training loss < 0.01 AND Rolling avg. of rel. change in training loss < 0.01 (window size 10). OR 1000 epochs.

### Model Evaluation
+ The model $M$ prediction is $\mathbb{1}[M(x)>0.5]$. Test accuracy is the percentage of correct predictions over the test set.

### Search algorithm

We seek the number of training samples needed to reach a fixed test accuracy using a kind of bisection algorithm.
1. Initial training run on 5000 samples.
    + If test accuracy > 0.9, take half step towards 0 -> 2'500
    + If test accuracy <= 0.9 take half step towards 10'000 -> 7500
2. Repeat, next with quarter step.

This is repeated $10$ times with different weight initialisations in case the test-accuracy curves are not monotonically increasing due to noise.

In [8]:
import numpy as np
import torch.nn as nn

1. [Why Are Convolutional Nets More Sample-Efficient than Fully-Connected Nets?](https://arxiv.org/abs/2010.08515) Zhiyuan Li, Yi Zhang, Sanjeev Arora, 2021